# Phase 3: Feature Engineering
**Project:** National Rent Intelligence Engine
**Goal:** Transform raw statistics into "Investor Intelligence" (Yields, Trends, and Pressure Signals).

**Key Features Created:**
1.  **Cap Rate (Yield):** The gross ROI of a property (`Annual Rent / Price`).
2.  **Lags (Momentum):** What was the rent 1 year ago? 2 years ago?
3.  **Student Pressure:** The gap between Student Visa growth and Housing Supply growth.
4.  **Target Variable:** We shift rent *backwards* by 1 year to teach the AI to predict the future.

In [2]:
# =========================================================
# PHASE 3: FEATURE ENGINEERING
# =========================================================
# Goal: Transform raw stats into "Investor Intelligence" (Yields, Trends, Pressure)
# Input: national_master_dataset.csv
# Output: national_training_data.csv (2019-2024), national_forecast_input.csv (2025)

import pandas as pd
import numpy as np
import os

# 1. SETUP PATHS
if os.path.exists("data/processed"):
    PROCESSED_PATH = "data/processed"
elif os.path.exists("../data/processed"):
    PROCESSED_PATH = "../data/processed"
else:
    PROCESSED_PATH = "../data/processed"
    os.makedirs(PROCESSED_PATH, exist_ok=True)

# 2. LOAD DATA
try:
    df = pd.read_csv(f"{PROCESSED_PATH}/national_master_dataset.csv")
    print(f"✅ Loaded Master Data: {len(df)} rows")
    print(f"   Columns: {df.columns.tolist()}")
except FileNotFoundError:
    print("❌ Error: Master dataset not found. Run Phase 2 first.")

# ---------------------------------------------------------
# 3. DATA CLEANING (Imputation)
# ---------------------------------------------------------
# We need continuous data for lags. We fill small gaps using linear interpolation.
df = df.sort_values(['City', 'Year'])

# Interpolate missing values within a city's timeline (e.g., missing 2021 in a 2019-2025 series)
cols_to_interpolate = ['Turnover_Rate', 'Total_Units', 'Intl_Students_Prov', 'Buy_Price']

for col in cols_to_interpolate:
    if col in df.columns:
        df[col] = df.groupby('City')[col].transform(lambda x: x.interpolate(method='linear', limit_direction='both'))

# Drop rows that are still hopelessly empty (usually cities with NO price data at all)
df = df.dropna(subset=['Average rent ($)', 'Buy_Price'])
print(f"✅ Data Cleaned. Rows remaining: {len(df)}")

# ---------------------------------------------------------
# 4. FINANCIAL FEATURES (The "Investor Mind")
# ---------------------------------------------------------
# Convert Rent to Annual
df['Annual_Rent'] = df['Average rent ($)'] * 12

# A) Cap Rate (Gross Yield)
# Formula: (Annual Rent / Buy Price) * 100
df['Cap_Rate'] = (df['Annual_Rent'] / df['Buy_Price']) * 100

# B) Price-to-Rent Ratio (Valuation)
df['Price_to_Rent_Ratio'] = df['Buy_Price'] / df['Annual_Rent']

✅ Loaded Master Data: 161 rows
   Columns: ['City', 'Turnover_Rate', 'Average rent ($)', 'Year', 'Total_Units', 'Buy_Price', 'Intl_Students_Prov']
✅ Data Cleaned. Rows remaining: 98


In [3]:
# ---------------------------------------------------------
# 5. PREDICTIVE FEATURES (Trends & Lags)
# ---------------------------------------------------------
# The AI needs to know "What happened last year?" to predict next year.

# A) Lags (Value 1 year ago, 2 years ago)
metrics = ['Average rent ($)', 'Turnover_Rate', 'Total_Units', 'Intl_Students_Prov', 'Cap_Rate']

for col in metrics:
    if col in df.columns:
        # Lag 1 (Last Year)
        df[f'{col}_Lag1'] = df.groupby('City')[col].shift(1)
        # Lag 2 (Two Years Ago)
        df[f'{col}_Lag2'] = df.groupby('City')[col].shift(2)
        
        # Growth Rate (%)
        df[f'{col}_Growth'] = df.groupby('City')[col].pct_change() * 100

# B) The "Pressure Cooker" Metric
# Demand Growth (Students) minus Supply Growth (Units)
# Positive = Shortage (Rents Go Up), Negative = Oversupply (Rents Stabilize)
if 'Intl_Students_Prov_Growth' in df.columns and 'Total_Units_Growth' in df.columns:
    df['Student_Supply_Mismatch'] = df['Intl_Students_Prov_Growth'] - df['Total_Units_Growth']
else:
    df['Student_Supply_Mismatch'] = 0 

# ---------------------------------------------------------
# 6. CREATE TARGET VARIABLE (The "Golden Column")
# ---------------------------------------------------------
# We want to predict Next Year's Rent.
# We shift rent BACKWARDS by 1 year. The 2023 row will get 2024's rent as the "Target".

df['Target_Next_Year_Rent'] = df.groupby('City')['Average rent ($)'].shift(-1)

# Calculate the Target Growth % (This is easier for AI to predict than raw dollars)
df['Target_Next_Year_Growth'] = (df['Target_Next_Year_Rent'] - df['Average rent ($)']) / df['Average rent ($)'] * 100

# ---------------------------------------------------------
# 7. SPLIT & SAVE
# ---------------------------------------------------------
# We remove the first 2 years (2019-2020) because they have empty Lags (NaNs)
df_model_ready = df.dropna(subset=['Average rent ($)_Lag2'])

# A) Training Data (2021-2024) - Rows where we KNOW the answer (Target is present)
df_train = df_model_ready.dropna(subset=['Target_Next_Year_Rent']).copy()

# B) Forecast Input (2025) - The most recent data (Target is NaN because 2026 hasn't happened yet)
# We will ask the AI to predict the missing target for these rows!
df_forecast = df_model_ready[df_model_ready['Target_Next_Year_Rent'].isna()].copy()

# Save
train_file = f"{PROCESSED_PATH}/national_training_data.csv"
forecast_file = f"{PROCESSED_PATH}/national_forecast_input.csv"

df_train.to_csv(train_file, index=False)
df_forecast.to_csv(forecast_file, index=False)

print("\n🎉 SUCCESS! Feature Engineering Complete.")
print(f"1. Training Data Saved: {len(df_train)} rows (Years: {df_train['Year'].unique()})")
print(f"2. Forecast Data Saved: {len(df_forecast)} rows (Year: {df_forecast['Year'].unique()})")
print(f"   -> Ready to predict 2026 rents for cities like: {df_forecast['City'].unique()[:5]}")
print("\n--- Preview of Training Data ---")
display(df_train[['City', 'Year', 'Average rent ($)', 'Cap_Rate', 'Target_Next_Year_Rent']].head())


🎉 SUCCESS! Feature Engineering Complete.
1. Training Data Saved: 56 rows (Years: [2021 2022 2023 2024])
2. Forecast Data Saved: 14 rows (Year: [2025])
   -> Ready to predict 2026 rents for cities like: <StringArray>
['Calgary', 'Edmonton', 'Halifax', 'Hamilton', 'London']
Length: 5, dtype: str

--- Preview of Training Data ---


,City,Year,Average rent ($),Cap_Rate,Target_Next_Year_Rent
54,Calgary,2021,1355.0,6.709074,1466.0
77,Calgary,2022,1466.0,6.635778,1695.0
100,Calgary,2023,1695.0,6.852330,1882.0
125,Calgary,2024,1882.0,6.696847,1914.0
55,Edmonton,2021,1270.0,8.714381,1304.0
